In [ ]:
import pandas as pd
from itertools import combinations
from collections import defaultdict
from blocking_utils.blocking_utils import compute_similarity
from datasketch import MinHash, MinHashLSH
from nltk.util import ngrams
import multiprocessing as mp
import logging
import time
import numpy as np
from tqdm import tqdm
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
df = None

In [ ]:
%%capture
global df
start_total = time.time()
df = pd.read_csv("data/processed/external_parties_train.csv")





: 

## Overview

block diagram with overview

## Feature Engineering

## Locality Sensitive Hashing

hash functions

parameters

elements in the same name bucket all have a very high similarity score
    F1 score at .76 regardless of threshold
elements in the same street name bucket have wide ranging similarity scores
    F1 score at .55 with threshold 0.1, .83 with threshold .6, .81 with .7

run-time: O(buckets * size_of_bucket^2)

max block size
- higher threshold
- prune blocks too big

Trade-off
few and small buckets: FAST!, higher precision lower recall
more buckets, bigger buckets: slooow, lower precision higher recall

## Entity Similarity

threshold, no LLMs, strictness trade-off -> more precision less recall 

## Evaluation

We evaluated our models on the training data. We do so by by looking at each pair of elements and label our prediction as a: 
* **True Positive**: $i$ and $j$ are the same entity and we predicted so.
* **False Positive**: we predicted $i$ and $j$ as the same entity, which is false.
* **False Negative**: we did not identify $i$ and $j$ as the same entity.

The evaluation metrics we used to guide our modeling are **Precision** and **Recall**